In [ ]:
import os
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
import sklearn
## can we please make this a relative path
DATABASE_DIR = "/Users/dsurrao/Documents/cfb/test/swdis"

# Allow us to display all the columns in a dataframe
pd.options.display.max_columns = None

We start by pulling in ALL the data to prep

In [ ]:
# Read in the data from the VIOLATION CSV file
def read_in_dataframes(root_dir):
    """Read in the SDWIS tables into dataframes and save them in a dictionary"""
    tables = ['ENFORCEMENT_ACTION',
              'GEOGRAPHIC_AREA',
              'LCR_SAMPLE',
              'LCR_SAMPLE_RESULT',
              'SERVICE_AREA',
              'TREATMENT',
              'VIOLATION',
              'VIOLATION_ENF_ASSOC',
              'WATER_SYSTEM',
              'WATER_SYSTEM_FACILITY']
    df = {}
    for table in tables:
        df[table] = pd.read_csv("{0}{1}{2}.csv".format(root_dir, os.path.sep, table),
                                index_col=False,
                                error_bad_lines=False)
    
    return df

df = read_in_dataframes(DATABASE_DIR)

In [ ]:
df['WATER_SYSTEM'].head()

In [ ]:
water_system_df = df['WATER_SYSTEM'][['EPA_REGION', 'POPULATION_SERVED_COUNT', 'PRIMACY_TYPE', 'PRIMARY_SOURCE_CODE',
                                  'PWS_ACTIVITY_CODE', 'PWSID', 'PWS_TYPE_CODE', 'SERVICE_CONNECTIONS_COUNT',
                                  'STATE_CODE', 'ZIP_CODE']].copy()

water_system_facility_df = df['WATER_SYSTEM_FACILITY'][['AVAILABILITY_CODE', 'FACILITY_ACTIVITY_CODE', 'FACILITY_DEACTIVATION_DATE',
                             'FACILITY_ID', 'FACILITY_NAME', 'FACILITY_TYPE_CODE', 'FILTRATION_STATUS_CODE', 
                             'IS_SOURCE_IND', 'IS_SOURCE_TREATED_IND', 'PRIMACY_AGENCY_CODE', 'PWSID', 
                             'SELLER_PWSID', 'SELLER_PWS_NAME', 'SELLER_TREATMENT_CODE', 'STATE_FACILITY_ID', 
                             'SUBMISSION_STATUS_CODE', 'WATER_TYPE_CODE']].copy()

#treatement_df = df['TREATMENT'][['COMMENTS_TEXT', 'FACILITY_ID', 'PWSID', 'TREATMENT_ID', 'TREATMENT_OBJECTIVE_Code',
#                                     'TREATMENT_PROCESS_CODE']]

violation_df = df['VIOLATION'][['COMPLIANCE_STATUS_CODE', 'COMPL_PER_BEGIN_DATE', 'COMPL_PER_END_DATE',
                                'CONTAMINANT_CODE', 'CORRECTIVE_ACTION_ID', 'FACILITY_ID',
                                'IS_HEALTH_BASED_IND', 'IS_MAJOR_VIOL_IND', 'LATEST_ENFORCEMENT_ID',
                                'ORIGINATOR_CODE', 'POPULATION_SERVED_COUNT', 'POP_CAT_5_CODE',
                                'PRIMACY_AGENCY_CODE', 'PRIMARY_SOURCE_CODE', 'PUBLIC_NOTIFICATION_TIER',
                                'PWSID', 'RTC_DATE', 'RTC_ENFORCEMENT_ID', 'RULE_CODE', 'RULE_FAMILY_CODE',
                                'RULE_GROUP_CODE', 'SEVERITY_IND_CNT', 'STATE_MCL', 'UNIT_OF_MEASURE',
                                'VIOLATION_CATEGORY_CODE', 'VIOLATION_CODE', 'VIOLATION_ID', 'VIOL_MEASURE']].copy()

enforcment_df = df['ENFORCEMENT_ACTION'][['ENFORCEMENT_ACTION_TYPE_CODE', 'ENFORCEMENT_COMMENT_TEXT',
                                          'ENFORCEMENT_DATE', 'ENFORCEMENT_ID', 'ORIGINATOR_CODE','PWSID']].copy()

violation_enf_assoc_df = df['VIOLATION_ENF_ASSOC'][['ENFORCEMENT_ID', 'PWSID', 'VIOLATION_ID']].copy()

geographic_area_df = df['GEOGRAPHIC_AREA'][['AREA_TYPE_CODE', 'CITY_SERVED', 'COUNTY_SERVED',
                                            'GEO_ID', 'PRIMACY_AGENCY_CODE', 'PWSID', 'STATE_SERVED',
                                            'TRIBAL_CODE', 'ZIP_CODE_SERVED']].copy()

service_area_df = df['SERVICE_AREA'][['IS_PRIMARY_SERVICE_AREA_CODE', 'PRIMACY_AGENCY_CODE',
                                      'PWSID','SERVICE_AREA_TYPE_CODE']].copy()

#new_dataframe.head()

In [ ]:
## merge water system facility subset to water system by pswid
data_set = pd.merge(water_system_df, water_system_facility_df, on=['PWSID'], validate="1:m")

## merge treatment to aggregate data set by pwsid
data_set = pd.merge(data_set, violation_df, on=['PWSID'])

## enforcemnt action to data set by pwsid
#data_set = pd.merge(data_set, enforcment_df, on=['PWSID'])

## pull in the violation enf assoc table
#data_set = pd.merge(data_set, violation_enf_assoc_df, on=['PWSID'])

data_set.head()

In [ ]:
data_set.dtypes

In [ ]:
# SERVICE_CONNECTIONS_COUNT is of type object, need to convert to numeric before calling Logit
# cols=['POPULATION_SERVED_COUNT_x', 'SERVICE_CONNECTIONS_COUNT']

cols=['POPULATION_SERVED_COUNT_x']

def convert_value(x): 
    if x == 'Y':
        return 1
    else: 
        return 0

data_set['DEP'] = data_set['IS_HEALTH_BASED_IND'].apply(convert_value)


In [ ]:
X=data_set[cols]
y=data_set['DEP']

import statsmodels.api as sm

logit_model = sm.Logit(y,X)
result = logit_model.fit()
print(result.summary())